## Introduction

[Video Walkthrough](https://www.youtube.com/watch?v=0kpseJLbEP4&list=PLypX5sYuDqvrqsXTw876gGHosCKvK_7QS&index=7)

In this section of the course, we will focus on querying and searching data in KDB.AI tables. By the end of this notebook, you will have a thorough understanding of the following:
- Selecting tables to query
- Performing queries and applying filters
- Customizing filters
- Conducting similarity searches
- Processing query results

### Setup

Install kdbai_client and import the necessary dependencies

In [ ]:
!pip install kdbai_client fastembed

In [ ]:
import kdbai_client as kdbai
import time
import pandas as pd
import numpy as np
from fastembed import TextEmbedding
import os
import getpass

##### Connect to KDB.AI

In [ ]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else input("KDB.AI API key: ")
)

In [ ]:
session = kdbai.Session(endpoint=KDBAI_ENDPOINT, api_key=KDBAI_API_KEY)
database = session.database("default")

### Create Our Table and Insert Data

In [ ]:
try:
    database.table("data").drop() # Drop the table if it already exists
except kdbai.KDBAIException:
    pass

##### Define a Schema

In [ ]:
schema = [
    {'name': 'id', 'type': 'int32'},
    {'name': 'name', 'type': 'str'},
    {'name': 'age', 'type': 'int16'},
    {'name': 'city', 'type': 'str'},
    {'name': 'description', 'type': 'str'},
    {'name': 'embeddings', 'type': 'float32s'}
]
index_name = 'hnws_index'
indexes = [{'name': index_name, 'column': 'embeddings', 'type': 'hnsw', 'params': {'dims': 384}}]

In [ ]:
table = database.create_table("data", schema=schema, indexes=indexes)

# Generate real vectors using FastEmbed
descriptions = [
    "A passionate environmentalist with 5 years of experience in conservation projects and enjoys hiking and outdoor activities.",
    "A software engineer with 7 years of experience in full-stack development, living in London, who loves to cook Italian cuisine.",
    "A guitarist with over 10 years of experience performing at local cafes and enjoys reading science fiction.",
    "A data scientist in Tokyo with 4 years of experience in machine learning and a keen interest in AI research.",
    "An avid reader and travel blogger with 3 years of experience visiting and writing about historic sites around the world.",
    "A graphic designer based in Berlin with 8 years of experience and a talent for creating digital art.",
    "A high school teacher with 15 years of experience in education who loves cycling and participates in charity rides.",
    "A professional photographer with 6 years of experience specializing in wildlife photography.",
    "A fitness trainer with 5 years of experience who enjoys helping people achieve their health goals.",
    "A chef with 12 years of experience who runs a popular restaurant and enjoys experimenting with new recipes.",
    "A journalist with 9 years of experience writing about technology and enjoys exploring new gadgets.",
    "A musician with 20 years of experience who plays multiple instruments and performs in a jazz band.",
    "A software developer with 6 years of experience in creating mobile apps and enjoys coding challenges.",
    "An artist with 10 years of experience who paints abstract pieces and has exhibited in several galleries.",
    "A historian with 7 years of experience who loves researching and writing about ancient civilizations.",
    "A marketing manager with 8 years of experience in digital marketing and social media strategy.",
    "A nurse with 12 years of experience in emergency care and patient management.",
    "A financial analyst with 5 years of experience in investment banking and portfolio management.",
    "A project manager with 10 years of experience in IT project coordination and execution.",
    "A UX designer with 6 years of experience in creating user-friendly interfaces for web and mobile applications.",
    "A sales executive with 8 years of experience in B2B sales and client relationship management.",
    "A content writer with 5 years of experience in creating engaging articles and blog posts.",
    "A civil engineer with 10 years of experience in infrastructure development and urban planning.",
    "A teacher with 15 years of experience in primary education and curriculum development.",
    "A business analyst with 7 years of experience in business process optimization and data analysis.",
    "A psychologist with 6 years of experience in clinical practice and mental health counseling.",
    "A software architect with 9 years of experience in designing scalable software solutions.",
    "A research scientist with 8 years of experience in biotechnology and genetic engineering.",
    "An operations manager with 12 years of experience in supply chain management and logistics.",
    "A public relations specialist with 7 years of experience in media relations and corporate communications."
]


##### Define an Embedding Model and Embed People Data

In [ ]:
embedding_model = TextEmbedding()
embeddings = list(embedding_model.embed(descriptions))

In [ ]:
import random
random.seed(42) # for reproducibility

names = ["Alice", "Bob", "Charlie", "Monica", "Eve", "Frank", "Grace", "Hannah", "Ivy", "Jack", "Kara", "Leo", "Mia", "Nate", "Olivia", "Paul", "Quinn", "Rita", "Sam", "Tina", "Uma", "Victor", "Wendy", "Xander", "Yara", "Zane", "Alice", "Cody", "Diana", "Ethan"]
cities = ["New York", "London", "New York", "Paris", "Berlin", "New York", "San Francisco", "Amsterdam", "Rome", "Toronto", "Chicago", "Barcelona", "Madrid", "New York", "Moscow", "Dubai", "Singapore", "New York", "Istanbul", "Munich", "Vienna", "Dublin", "Zurich", "Stockholm", "Lisbon", "Prague", "Budapest", "Berlin", "Copenhagen", "Seoul"]


data = pd.DataFrame({
    'id': np.array(list(range(0, 30)), dtype='int32'),
    'name': names,
    'age': np.array([random.randint(18, 60) for _ in range(30)], dtype='int16'),
    'city': cities,
    'description': descriptions,
    'embeddings': embeddings
})

##### Insert the Data

In [ ]:
table.insert(data)

True

### Query Data

In [ ]:
print("All data in the table:")
table.query()

All data in the table:


,id,name,age,city,description,embeddings
0,0,Alice,58,New York,A passionate environmentalist with 5 years of ...,"[-0.006158471, 0.063678846, 0.09181005, -0.023..."
1,1,Bob,25,London,A software engineer with 7 years of experience...,"[-0.035581246, 0.07986437, 0.04891828, -0.0604..."
2,2,Charlie,19,New York,A guitarist with over 10 years of experience p...,"[0.050266247, 0.05255312, 0.048840936, -0.0032..."
3,3,Monica,35,Paris,A data scientist in Tokyo with 4 years of expe...,"[-0.008097345, 0.030305384, 0.012246384, -0.04..."
4,4,Eve,33,Berlin,An avid reader and travel blogger with 3 years...,"[0.029772803, 0.07571457, 0.042140756, 0.06809..."
5,5,Frank,32,New York,A graphic designer based in Berlin with 8 year...,"[0.013257692, 0.045190323, 0.0074770325, -0.00..."
6,6,Grace,26,San Francisco,A high school teacher with 15 years of experie...,"[-0.011028861, 0.051242497, 0.063257486, -0.05..."
7,7,Hannah,24,Amsterdam,A professional photographer with 6 years of ex...,"[0.04469839, 0.07050187, 0.046390466, -0.03404..."
8,8,Ivy,52,Rome,A fitness trainer with 5 years of experience w...,"[0.0002550126, 0.024398372, 0.09861772, 0.0062..."
9,9,Jack,23,Toronto,A chef with 12 years of experience who runs a ...,"[-0.008186043, 0.051337104, 0.02683556, -0.030..."


##### Query with Filters

In [ ]:
print("Querying data where age >= 30 and city is Rome or Paris:")
table.query(filter=[(">=", "age", 30), ("in", "city", ["Rome", "Paris"])])

Querying data where age >= 30 and city is Rome or Paris:


,id,name,age,city,description,embeddings
0,3,Monica,35,Paris,A data scientist in Tokyo with 4 years of expe...,"[-0.008097345, 0.030305384, 0.012246384, -0.04..."
1,8,Ivy,52,Rome,A fitness trainer with 5 years of experience w...,"[0.0002550126, 0.024398372, 0.09861772, 0.0062..."


In [ ]:
print("Aggregated query for names and cities:")
print(table.query(aggs={"name": "name", "city": "city"})) # returns only the names and cities

Aggregated query for names and cities:
       name           city
0     Alice       New York
1       Bob         London
2   Charlie       New York
3    Monica          Paris
4       Eve         Berlin
5     Frank       New York
6     Grace  San Francisco
7    Hannah      Amsterdam
8       Ivy           Rome
9      Jack        Toronto
10     Kara        Chicago
11      Leo      Barcelona
12      Mia         Madrid
13     Nate       New York
14   Olivia         Moscow
15     Paul          Dubai
16    Quinn      Singapore
17     Rita       New York
18      Sam       Istanbul
19     Tina         Munich
20      Uma         Vienna
21   Victor         Dublin
22    Wendy         Zurich
23   Xander      Stockholm
24     Yara         Lisbon
25     Zane         Prague
26    Alice       Budapest
27     Cody         Berlin
28    Diana     Copenhagen
29    Ethan          Seoul


In [ ]:
print("Aggregated query for maximum age grouped by city:")
print(table.query(aggs={'maxAge': ['max', 'age']}, group_by=['city'], sort_columns=['maxAge']))

Aggregated query for maximum age grouped by city:
             city  maxAge
0           Seoul      18
1          Munich      19
2          Madrid      20
3          Moscow      23
4         Toronto      23
5       Amsterdam      24
6          London      25
7   San Francisco      26
8          Dublin      30
9           Dubai      31
10         Prague      32
11      Singapore      32
12     Copenhagen      35
13          Paris      35
14         Lisbon      44
15      Barcelona      45
16       Budapest      46
17           Rome      52
18      Stockholm      52
19         Vienna      53
20         Berlin      55
21        Chicago      55
22       Istanbul      56
23       New York      58
24         Zurich      59


In [ ]:
print("Aggregated query for average age and count of distict people in each city:")
table.query(aggs={'avgAge': ['avg', 'age'], 'countCity': ['count', 'id']}, group_by=['city'], sort_columns=['avgAge'])

Aggregated query for average age and count of distict people in each city:


,city,avgAge,countCity
0,Seoul,18.0,1
1,Munich,19.0,1
2,Madrid,20.0,1
3,Moscow,23.0,1
4,Toronto,23.0,1
5,Amsterdam,24.0,1
6,London,25.0,1
7,San Francisco,26.0,1
8,Dublin,30.0,1
9,Dubai,31.0,1


##### Customizing Filters

In [ ]:
print("Querying data where age < 30 and name starts with H:")
print(table.query(filter=[("<", "age", 30), ("like", "name", "H*")]))

Querying data where age < 30 and name starts with H:
   id    name  age       city  \
0   7  Hannah   24  Amsterdam   

                                         description  \
0  A professional photographer with 6 years of ex...   

                                          embeddings  
0  [0.04469839, 0.07050187, 0.046390466, -0.03404...  


In [ ]:
print("Querying data where age > 30 and city is Rome or New York:")
print(table.query(filter=[(">", "age", 30), ("in", "city", ["Rome", "New York"])]))

Querying data where age > 30 and city is Rome or New York:
   id   name  age      city  \
0   0  Alice   58  New York   
1   5  Frank   32  New York   
2   8    Ivy   52      Rome   
3  17   Rita   50  New York   

                                         description  \
0  A passionate environmentalist with 5 years of ...   
1  A graphic designer based in Berlin with 8 year...   
2  A fitness trainer with 5 years of experience w...   
3  A financial analyst with 5 years of experience...   

                                          embeddings  
0  [-0.006158471, 0.063678846, 0.09181005, -0.023...  
1  [0.013257692, 0.045190323, 0.0074770325, -0.00...  
2  [0.0002550126, 0.024398372, 0.09861772, 0.0062...  
3  [0.015000028, 0.024906091, 0.0010010687, 0.011...  


#### Vector Search

##### Embedding a Query Vector and Searching

In [ ]:
person_query = "a software engineer with lots of experience"

In [ ]:
person_embedding = list(embedding_model.embed([person_query]))[0].tolist()

In [ ]:
len(person_embedding)

384

In [ ]:
print("Searching for the three closest people to the example vector:")
table.search({index_name: [person_embedding]}, n=3)

Searching for the three closest people to the example vector:


[   id   name  age      city  \
 0  26  Alice   46  Budapest   
 1  12    Mia   20    Madrid   
 2  19   Tina   19    Munich   
 
                                          description  \
 0  A software architect with 9 years of experienc...   
 1  A software developer with 6 years of experienc...   
 2  A UX designer with 6 years of experience in cr...   
 
                                           embeddings  __nn_distance  
 0  [-0.06051296, 0.031862404, -0.031203829, -0.07...       0.322560  
 1  [-0.04372146, 0.06704399, 0.022140108, -0.1017...       0.356629  
 2  [-0.030572662, 0.04520395, 0.04553928, -0.0925...       0.457364  ]

##### Batch Search with Multiple Query Vectors

In [ ]:
print("Batch search with multiple query vectors:")
queries = ["a software engineer with lots of experience", "a data scientist with experience in machine learning and a keen interest in AI research"]
queries_embeddings = list(embedding_model.embed(queries))
table.search(vectors={index_name: [q.tolist() for q in queries_embeddings]}, n=3)

Batch search with multiple query vectors:


[   id   name  age      city  \
 0  26  Alice   46  Budapest   
 1  12    Mia   20    Madrid   
 2  19   Tina   19    Munich   
 
                                          description  \
 0  A software architect with 9 years of experienc...   
 1  A software developer with 6 years of experienc...   
 2  A UX designer with 6 years of experience in cr...   
 
                                           embeddings  __nn_distance  
 0  [-0.06051296, 0.031862404, -0.031203829, -0.07...       0.322560  
 1  [-0.04372146, 0.06704399, 0.022140108, -0.1017...       0.356629  
 2  [-0.030572662, 0.04520395, 0.04553928, -0.0925...       0.457364  ,
    id    name  age    city                                        description  \
 0   3  Monica   35   Paris  A data scientist in Tokyo with 4 years of expe...   
 1  24    Yara   44  Lisbon  A business analyst with 7 years of experience ...   
 2  27    Cody   55  Berlin  A research scientist with 8 years of experienc...   
 
                         

##### Combining Aggregations with Vector Search

In [ ]:
print("Searching with aggregated results for name, city, and description:")
print(table.search(vectors={index_name: [person_embedding]}, n=3, aggs={"name": "name", "city": "city", "description": "description"}))

Searching with aggregated results for name, city, and description:
[    name      city                                        description
0  Alice  Budapest  A software architect with 9 years of experienc...
1    Mia    Madrid  A software developer with 6 years of experienc...
2   Tina    Munich  A UX designer with 6 years of experience in cr...]


##### Vector Search with Filters

In [ ]:
print("Searching with filter to find people younger than 30:")
print(table.search(vectors={index_name: [person_embedding]}, n=5, filter=[("<", "age", 30)], aggs={"name": "name", "age": "age", "description": "description"}))

Searching with filter to find people younger than 30:
[    name  age                                        description
0    Mia   20  A software developer with 6 years of experienc...
1   Tina   19  A UX designer with 6 years of experience in cr...
2    Bob   25  A software engineer with 7 years of experience...
3  Ethan   18  A public relations specialist with 7 years of ...
4   Jack   23  A chef with 12 years of experience who runs a ...]


### Drop Table To Conserve Resources

In [ ]:
table.drop()

True